Урок 3. MongoDB

In [1]:
import requests
from bs4 import BeautifulSoup as bs   # у нас html странички, потому берем это     
from pprint import pprint
import time
import pandas as pd

#1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, 
#   записывающую собранные вакансии в созданную БД
#2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы
# Поскольку я предыдущее задание делал для базы продуктов - работать с Монго буду по продуктам!!

head ={'User-Agent': 'YaBrowser/19.7.5.90.01'}
main_link = 'https://roscontrol.com/category/produkti/'
site_name = 'https://roscontrol.com/'
forbidden_page = 'https://roscontrol.com/product/produkti/'
lst =[]
# по категории [1]:
time.sleep(0.2); req2 = requests.get(main_link).text
parsed_req2 = bs(req2,'html.parser')
block_2 = parsed_req2.find('div',{'class':'grid-row grid-flex-mobile'})
list_categories = block_2.findAll('a')
for en_categories in list_categories:
    categorie_name = en_categories['href'].split('/')[3]
    categorie_link = main_link + categorie_name + '/'
    
    time.sleep(0.2); req2a = requests.get(categorie_link).text
    parsed_req2a = bs(req2a,'html.parser')
    block_2a = parsed_req2a.find('div',{'class':'grid-row grid-flex-mobile'})
    if(block_2a==None): break # если список закончился, а теги еще остались...
    list_goods = block_2a.findAll('a')
    for en_goods in list_goods:
        good_name = en_goods['href'].split('/')[4]
        good_link = categorie_link + good_name + '/'
        
        time.sleep(0.2); req2b = requests.get(good_link).text
        parsed_req2b = bs(req2b,'html.parser')
        block_2b = parsed_req2b.find('div',{'class':'AJAX_root'})
        if(block_2b==None): break # если список закончился, а теги еще остались...
        list_points = block_2b.findAll('a')
        
        for en_points in list_points:
            point_name = en_points['href'].split('/')[2]
            point_link = 'https://roscontrol.com/product/' + point_name + '/'
            print(point_link)
            if point_link == forbidden_page: continue                # пустая карточка продукта
            time.sleep(0.2); req2c = requests.get(point_link).text
            parsed_req2c = bs(req2c,'html.parser')
            dngr_rate,qual_rate,total_rate = '*','*','*'

            q_block = parsed_req2c.find('div',{'id':'product__single-rev-rating'})
            if (q_block==None): continue         # страница не найдена!!
            q_total = q_block.find('div',{'id':'product__single-rev-total'})
            green   = q_total.find('div',{'class':'total green'})
            if(not(green==None)):  total_rate = green.string
#
            q_rates = q_block.find('div',{'id':'product__single-rev-rate'})
#
            grp_all  = q_rates.findAll('div',{'class':'rate-item group'})
            if(len(grp_all)>0):
                dngr = grp_all[0].find('div',{'class':'rate-item__value'})
                dngr_tag = dngr.find('span')
                dngr_rate = dngr_tag.string
    #
                if(len(grp_all)>3):
                    qual = grp_all[3].find('div',{'class':'rate-item__value'})
                    qual_tag = qual.find('span')
                    qual_rate = qual_tag.string

            
            lst.append([point_name,categorie_name,good_name,dngr_rate,qual_rate,total_rate,site_name])
            if(len(lst)>50): break
        if(len(lst)>50): break
    if(len(lst)>50): break

#DF = pd.DataFrame(lst,columns=['point','category','sub_ctgr','dangrless','quality','rating','site'])
#DF
#

https://roscontrol.com/product/moloko-ekomilk/
https://roscontrol.com/product/krasnaya-tsena-32-ultrapasterizovannoe/
https://roscontrol.com/product/moloko-prostokvashino-3-2/
https://roscontrol.com/product/selo-zelenoe-32-ultrapasterizovannoe/
https://roscontrol.com/product/prostokvashino-otbornoe-ot-34-do-45-pasterizovannoe/
https://roscontrol.com/product/ostankinskoe-32-ultrapasterizovannoe/
https://roscontrol.com/product/sarafanovo-32-ultrapasterizovannoe/
https://roscontrol.com/product/valio-32-ultrapasterizovannoe/
https://roscontrol.com/product/avida-32-ultrapasterizovannoe/
https://roscontrol.com/product/vkusnotieievo-3-2-pastierizovannoie/
https://roscontrol.com/product/asenevskaya-ferma-32-pasterizovannoe/
https://roscontrol.com/product/ruzskoe-32-4-pasterizovannoe-tselnoe/
https://roscontrol.com/product/domik-v-derevne-35-tselnoe-pasterizovannoe/
https://roscontrol.com/product/mamulya-32-ultrapasterizovannoe/
https://roscontrol.com/product/vologganka-15-pasterizovannoe/
http

In [2]:
from pymongo import MongoClient
client = MongoClient('localhost',27017)
db = client['less3']
coll = db.products
n = len(lst)
for i in range(n):
    wrd3,wrd4,wrd5 = lst[i][3:6]
    if wrd3=='*': wrd3=0
    if wrd4=='*': wrd4=0
    if wrd5=='*': wrd5=0
    rec = {'good': lst[i][0],'catg': lst[i][1],'sub_catg': lst[i][2],'dngrls': int(wrd3),'qual': int(wrd4),'rate': int(wrd5),'link': lst[i][6]}
    coll.insert_one(rec)

In [3]:
find_list = coll.find({'rate': {'$gt': 80}})
for en in list(find_list):
    print('-----------')
    print('product:        {}'.format(en['good']))
    print('category:       {}'.format(en['catg']))
    print('sub_cat:        {}'.format(en['sub_catg']))
    print('class dngrless: {}'.format(en['dngrls']))
    print('class quality:  {}'.format(en['qual']))
    print('rating:         {}'.format(en['rate']))
    print('link:           {}'.format(en['link']))


-----------
product:        prostokvashino-otbornoe-ot-34-do-45-pasterizovannoe
category:       molochnie_produkti
sub_cat:        moloko
class dngrless: 92
class quality:  91
rating:         82
link:           https://roscontrol.com/
-----------
product:        kefir-asenevskaya-ferma-32
category:       molochnie_produkti
sub_cat:        kefir
class dngrless: 88
class quality:  91
rating:         83
link:           https://roscontrol.com/
-----------
product:        kefir-nasha-semya-lakomo
category:       molochnie_produkti
sub_cat:        kefir
class dngrless: 94
class quality:  78
rating:         81
link:           https://roscontrol.com/
-----------
product:        kefir-prostokvashino
category:       molochnie_produkti
sub_cat:        kefir
class dngrless: 88
class quality:  89
rating:         81
link:           https://roscontrol.com/
-----------
product:        kefir-biomax-1
category:       molochnie_produkti
sub_cat:        kefir
class dngrless: 95
class quality:  72
rating: 

In [ ]:
#db.products.drop()